In [1]:
import re
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [172]:
#txt_file = 'eng_elec_text.txt'
#txt_file = 'eng_elec_bigram.txt'
#txt_file = 'eng_folk_text.txt'
#txt_file = 'eng_folk_bigram.txt'
#txt_file = 'eng_hiphop_text.txt'
txt_file = 'eng_hiphop_bigram.txt'
#txt_file = 'eng_rnb_text.txt'
#txt_file = 'eng_rnb_bigram.txt'
#txt_file = 'eng_rock_text.txt'
#txt_file = 'eng_rock_bigram.txt'
#txt_file = 'kor_ballad_text.txt'
#txt_file = 'kor_ballad_bigram.txt'
#txt_file = 'kor_dance_text.txt'
#txt_file = 'kor_dance_bigram.txt'
#txt_file = 'kor_folk_text.txt'
#txt_file = 'kor_folk_bigram.txt'
#txt_file = 'kor_hiphop_text.txt'
#txt_file = 'kor_hiphop_bigram.txt'
#txt_file = 'kor_indie_text.txt'
#txt_file = 'kor_indie_bigram.txt'
#txt_file = 'kor_rnb_text.txt'
#txt_file = 'kor_rnb_bigram.txt'
#txt_file = 'kor_rock_text.txt'
#txt_file = 'kor_rock_bigram.txt'
#txt_file = 'kor_trot_text.txt'
#txt_file = 'kor_trot_bigram.txt'
raw_corpus = []

with open(txt_file, 'r', encoding = 'utf-8') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)
print(len(raw_corpus))

258604


In [173]:
raw_corpus[:5]

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                         #소문자 변경 후 양쪽 공백 제거
    sentence = re.sub(r"([?.!,¿]).,", r" \1 ", sentence)          #특수문자 양쪽에 공백 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                 #여러 개의 공백이 붙어있으면 하나의 공백으로
    sentence = re.sub(r"[^a-zA-Z가-힣?!¿]+", " ", sentence)   #영어, 알파벳, ?, !, ¿ 제외 모두 공백으로
    sentence = sentence.strip()                                 #양쪽 공백 제거
    sentence = re.sub(r"\(.\)", " ", sentence)                  #괄호 제거
    sentence = '<start> ' + sentence + ' <end>'                 #start, end 추가
    return sentence

corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ':':
        continue
    if len(sentence)>150:
        continue

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 15000, filters = ' ', oov_token = '<unk>')
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    print('토크나이저: ', tokenizer, '\n', tensor)

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 1234)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epochs = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder = True)

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder = True)


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.lstm_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.lstm_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.lstm_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.lstm_1(out)
        out = self.lstm_2(out)
        out = self.lstm_3(out)
        out = self.dropout(out)
        out = self.linear(out)

        return out


embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)
model

for src_sample, tgt_sample in dataset_train.take(1): break
model(src_sample)

model.summary()

tf.test.is_gpu_available()

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        )
    except RuntimeError as e:
        print(e)

토크나이저:  <keras.src.preprocessing.text.Tokenizer object at 0x7f3ce0491f60> 
 [[  2 119   6 ...   0   0   0]
 [  2 208 258 ...   0   0   0]
 [  2   4  41 ...   0   0   0]
 ...
 [  2   6  95 ...   0   0   0]
 [  2   8  16 ...   0   0   0]
 [  2   8  16 ...   0   0   0]]
Model: "text_generator_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    multiple                  3840256   
                                                                 
 lstm_23 (LSTM)              multiple                  5246976   
                                                                 
 lstm_24 (LSTM)              multiple                  8392704   
                                                                 
 lstm_25 (LSTM)              multiple                  8392704   
                                                                 
 dropout_1 (Dropout)         multiple        

In [174]:
model.summary()

Model: "text_generator_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    multiple                  3840256   
                                                                 
 lstm_23 (LSTM)              multiple                  5246976   
                                                                 
 lstm_24 (LSTM)              multiple                  8392704   
                                                                 
 lstm_25 (LSTM)              multiple                  8392704   
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_11 (Dense)            multiple                  15376025  
                                                                 
Total params: 41248665 (157.35 MB)
Trainable para

In [175]:
model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset_train, epochs = 10)

Epoch 1/10
808/808 [==============================] - 126s 151ms/step - loss: 1.3481
Epoch 2/10
808/808 [==============================] - 123s 152ms/step - loss: 1.1411
Epoch 3/10
808/808 [==============================] - 123s 152ms/step - loss: 1.0826
Epoch 4/10
808/808 [==============================] - 124s 152ms/step - loss: 1.0398
Epoch 5/10
808/808 [==============================] - 123s 152ms/step - loss: 1.0048
Epoch 6/10
808/808 [==============================] - 124s 153ms/step - loss: 0.9715
Epoch 7/10
808/808 [==============================] - 124s 153ms/step - loss: 0.9418
Epoch 8/10
808/808 [==============================] - 124s 153ms/step - loss: 0.9142
Epoch 9/10
808/808 [==============================] - 124s 153ms/step - loss: 0.8888
Epoch 10/10
808/808 [==============================] - 124s 152ms/step - loss: 0.8652


In [176]:
def generate_text(model, tokenizer, init_sentence = '<start>', max_len = 30):
    #테스트를 위해 입력받은 init_sentence도 텐서 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype = tf.int64)
    end_token = tokenizer.word_index['<end>']

    #단어를 하나씩 예측해 문장 생성
    while True:
        #1. 입력받은 문장의 텐서 입력
        predict = model(test_tensor)
        #2. 예측된 값 중 가장 높은 확률인 word index를 출력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]
        #3. 2에서 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis = -1)
        #4. 모델이 <end>를 예측했거나 max_len에 도달하면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ''
    #tokenizer를 활용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + ' '

    return generated

In [186]:
generate_text(model, tokenizer, init_sentence = '<start> bitch')

'<start> bitch i m a boss <end> '